<a href="https://colab.research.google.com/github/kittisaktaoma/colab_practice/blob/master/tmp/caox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
import pandas as pd

In [44]:
df = pd.read_csv("drive/My Drive/jak3_chembl.txt")
df.head()

,CMPD_CHEMBLID\tMOLREGNO\tPARENT_CMPD_CHEMBLID\tPARENT_MOLREGNO\tMOL_PREF_NAME\tCOMPOUND_KEY\tMOLWEIGHT\tALOGP\tPSA\tNUM_RO5_VIOLATIONS\tCANONICAL_SMILES\tACTIVITY_ID\tSTANDARD_TYPE\tRELATION\tSTANDARD_VALUE\tSTANDARD_UNITS\tPCHEMBL_VALUE\tPUBLISHED_TYPE\tPUBLISHED_RELATION\tPUBLISHED_VALUE\tPUBLISHED_UNITS\tACTIVITY_COMMENT\tDATA_VALIDITY_COMMENT\tPOTENTIAL_DUPLICATE\tBAO_ENDPOINT\tUO_UNITS\tQUDT_UNITS\tASSAY_ID\tASSAY_CHEMBLID\tASSAY_TYPE\tDESCRIPTION\tASSAY_SRC_ID\tASSAY_SRC_DESCRIPTION\tASSAY_ORGANISM\tASSAY_STRAIN\tASSAY_TAX_ID\tCURATED_BY\tBAO_FORMAT\tTID\tTARGET_CHEMBLID\tTARGET_TYPE\tPROTEIN_ACCESSION\tPREF_NAME\tORGANISM\tCONFIDENCE_SCORE\tTARGET_MAPPING\tAPD_NAME\tAPD_CONFIDENCE\tDOC_ID\tDOC_CHEMBLID\tPUBMED_ID\tJOURNAL\tYEAR\tVOLUME\tISSUE\tFIRST_PAGE\tCELL_ID\tCELL_CHEMBL_ID\tCELL_NAME
0,CHEMBL1998121\t1319412\tCHEMBL1998121\t1319412...
1,CHEMBL1990254\t1311545\tCHEMBL1990254\t1311545...
2,CHEMBL1995813\t1317104\tCHEMBL1995813\t1317104...
3,CHEMBL1972158\t1293449\tCHEMBL1972158\t1293449...
4,CHEMBL1998829\t1320120\tCHEMBL1998829\t1320120...


In [21]:
#! git clone https://github.com/Superzchen/iFeature
! python iFeature/iFeature.py --file sample.csv --type AAC --out AAC.txt

The input file seems not in fasta format.
